In [3]:
import numpy as np
import math

In [4]:
#reading data points 1
pts = np.zeros((1,2))
i=0
with open(r'./SpectData.txt') as file:
    for readline in file:
        line_strip = readline.strip() # removing \n
        point = line_strip.split('\t') # removing \t

        if(i==0):
            pts[0]=point
        else:
            pts=np.append(pts,np.array([point]),axis=0)  
        i+=1             
data=pts.astype(float)

data[0]
len(data)

2084

In [ ]:
#reading data points 2
# pts2 = np.zeros((1,2))
# i=0
# with open(r'C:\Users\itisk\Desktop\File Storage\Workspace\1\ML\Data\SpectData2.txt') as file:
#     for readline in file:
#         line_strip = readline.strip() # removing \n
#         point = line_strip.split('\t') # removing \t

#         if(i==0):
#             pts2[0]=point
#         else:
#             pts2=np.append(pts2,np.array([point]),axis=0)  
#         i+=1             
# data2=pts.astype(float)

# data2[0]
# len(data2)

In [5]:
def weightFunc(pts,sigma):
    res=np.array([])
    for i in range(len(pts)):
        for j in range(i+1,len(pts)):
            calc=-1*(((pts[i][0]-pts[j][0])**2) + ((pts[i][1]-pts[j][1])**2))/2*((sigma*10**-2)**2)
            res=np.append(res,np.array([calc]),axis=0)
    print(sigma)
    res=np.exp(res)
    return(res)


In [8]:
for i in range(101):
    y = weightFunc(data[:200],i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [ ]:
print(len(y))
print("===============================================")
print(y)